# Website A/B Testing - Lab

## Introduction

In this lab, you'll get another chance to practice your skills at conducting a full A/B test analysis. It will also be a chance to practice your data exploration and processing skills! The scenario you'll be investigating is data collected from the homepage of a music app page for audacity.

## Objectives

You will be able to:
* Analyze the data from a website A/B test to draw relevant conclusions
* Explore and analyze web action data

## Exploratory Analysis

Start by loading in the dataset stored in the file 'homepage_actions.csv'. Then conduct an exploratory analysis to get familiar with the data.

> Hints:
    * Start investigating the id column:
        * How many viewers also clicked?
        * Are there any anomalies with the data; did anyone click who didn't view?
        * Is there any overlap between the control and experiment groups? 
            * If so, how do you plan to account for this in your experimental design?

In [101]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from statsmodels.formula.api import ols
import collections
import flatiron_stats as fs
import numpy as np

In [4]:
#Your code here
data = pd.read_csv('homepage_actions.csv')

In [5]:
data.head()

,timestamp,id,group,action
0,2016-09-24 17:42:27.839496,804196,experiment,view
1,2016-09-24 19:19:03.542569,434745,experiment,view
2,2016-09-24 19:36:00.944135,507599,experiment,view
3,2016-09-24 19:59:02.646620,671993,control,view
4,2016-09-24 20:26:14.466886,536734,experiment,view


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8188 entries, 0 to 8187
Data columns (total 4 columns):
timestamp    8188 non-null object
id           8188 non-null int64
group        8188 non-null object
action       8188 non-null object
dtypes: int64(1), object(3)
memory usage: 256.0+ KB


In [86]:
data.groupby(['id','timestamp']).first().head()

group action
id     timestamp                                    
182988 2016-11-26 14:38:47.682931  experiment   view
182994 2016-12-14 13:19:00.940677     control   view
       2016-12-14 13:20:29.564080     control  click
183089 2016-09-29 23:04:42.722325     control   view
183136 2016-12-27 10:13:58.808014  experiment   view

In [67]:
control = data[data['group']=='control']
experiment = data[data['group']=='experiment']


In [71]:
control_counts = control.groupby('id')['timestamp'].count()
control_2 = control_counts[control_counts ==2]
control_1 = control_counts[control_counts ==1]

control[control['id'].isin(control_1.index)].action.value_counts()

view    2400
Name: action, dtype: int64

In [73]:
control[control['id'].isin(control_2.index)].action.value_counts()

click    932
view     932
Name: action, dtype: int64

In [88]:
len(control_1)

2400

In [56]:
experiment_counts = experiment.groupby('id')['timestamp'].count()
experiment_2 = experiment_counts[experiment_counts ==2]
experiment_1 = experiment_counts[experiment_counts ==1]

experiment[experiment['id'].isin(experiment_1.index)].action.value_counts()

view    2068
Name: action, dtype: int64

In [57]:
experiment[experiment['id'].isin(experiment_2.index)].action.value_counts()

click    928
view     928
Name: action, dtype: int64

In [83]:
control_ids = set(control['id'])
experimental_ids = set(experiment['id'])
control_ids.intersection(experimental_ids)

set()

## Conduct a Statistical Test

Conduct a statistical test to determine whether the experimental homepage was more effective than that of the control group.

In [90]:
#Your code here

#H0: There is no difference between the click rate of the control site and the click
#    rate of the experimental site.  p(control) = p(experimental)
#HA: The click rate of the experimental site is greater than the click rate of the 
#    control site.  

data['count'] = 1
data.head()

,timestamp,id,group,action,count
0,2016-09-24 17:42:27.839496,804196,experiment,view,1
1,2016-09-24 19:19:03.542569,434745,experiment,view,1
2,2016-09-24 19:36:00.944135,507599,experiment,view,1
3,2016-09-24 19:59:02.646620,671993,control,view,1
4,2016-09-24 20:26:14.466886,536734,experiment,view,1


In [91]:
control = data[data.group=='control'].pivot(index='id', columns='action', values='count')
control = control.fillna(value=0)

experiment = data[data.group=='experiment'].pivot(index='id', columns='action', values='count')
experiment = experiment.fillna(value=0)



print("Sample sizes:\tControl: {}\tExperiment: {}".format(len(control), len(experiment)))
print("Total Clicks:\tControl: {}\tExperiment: {}".format(control.click.sum(), experiment.click.sum()))
print("Average click rate:\tControl: {}\tExperiment: {}".format(control.click.mean(), experiment.click.mean()))
control.head()

Sample sizes:	Control: 3332	Experiment: 2996
Total Clicks:	Control: 932.0	Experiment: 928.0
Average click rate:	Control: 0.2797118847539016	Experiment: 0.3097463284379172


action,click,view
id,,
182994,1.0,1.0
183089,0.0,1.0
183248,1.0,1.0
183515,0.0,1.0
183524,0.0,1.0


In [94]:
fs.p_value_welch_ttest(control.click, experiment.click)

0.004466402814337078

P_Val is less than the alpha threshold of 0.05 so we reject the null hypothesis that there is no difference between the websites.  


## Verifying Results

One sensible formulation of the data to answer the hypothesis test above would be to create a binary variable representing each individual in the experiment and control group. This binary variable would represent whether or not that individual clicked on the homepage; 1 for they did and 0 if they did not. 

The variance for the number of successes in a sample of a binomial variable with n observations is given by:

## $n\bullet p (1-p)$

Given this, perform 3 steps to verify the results of your statistical test:
1. Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 
2. Calculate the number of standard deviations that the actual number of clicks was from this estimate. 
3. Finally, calculate a p-value using the normal distribution based on this z-score.

### Step 1:
Calculate the expected number of clicks for the experiment group, if it had the same click-through rate as that of the control group. 

In [98]:
#Your code here
control_rate = control.click.mean()
expected_experiment_clicks_under_null = control_rate * len(experiment)
print(expected_experiment_clicks_under_null)

838.0168067226891


### Step 2:
Calculate the number of standard deviations that the actual number of clicks was from this estimate.

In [102]:
#Your code here
(experiment.click.sum()-expected_experiment_clicks_under_null)/np.sqrt(len(experiment)*control_rate*(1-control_rate))

3.6625360854823588

### Step 3: 
Finally, calculate a p-value using the normal distribution based on this z-score.

In [104]:
#Your code here
1-stats.norm.cdf(3.6625360854823588)

0.00012486528006949715



### Analysis:

Does this result roughly match that of the previous statistical test?

> Comment: It does match.  Both test show that we will reject the null hypothesis.  

## Summary

In this lab, you continued to get more practice designing and conducting AB tests. This required additional work preprocessing and formulating the initial problem in a suitable manner. Additionally, you also saw how to verify results, strengthening your knowledge of binomial variables, and reviewing initial statistical concepts of the central limit theorem, standard deviation, z-scores, and their accompanying p-values.